In [ ]:
import io
import re
import string
import tqdm
import numpy as np
import nltk
import tensorflow as tf
from tensorflow.keras import layers
import pandas as pd
from bs4 import BeautifulSoup
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize

In [ ]:
nltk.download('stopwords')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
## reading training data. This was used initally to test the accuracy using cross validation . Do not run this for the test data run
data=open("/content/drive/MyDrive/Training Data.txt","r")


In [ ]:
## reading training data. This was used initally to test the accuracy using cross validation . Do not run this for the test data run
data=data.readlines()

In [ ]:
## d is the array form of the reviews in the training data
d=[]

In [ ]:
## reading training data. This was used initally to test the accuracy using cross validation . Do not run this for the test data run
## reading only the reviews into d without the class identifiers +1/-1
for l in data:
    s=str(l[2:len(l)-6])
    d.append(s)


In [ ]:
## reading test data. Run this for the test data run
test_data=open("/content/drive/MyDrive/TestData.txt","r")
test_data=test_data.readlines()
## td is the array form of the reviews in the test data
td=[]
for l in test_data:
    s=str(l[2:len(l)-6])
    td.append(s)

In [ ]:
## y holds the class data of each review in the training data 
y=[]

In [ ]:
for l in data:
    s=int(l[0:2])
    y.append(s)

In [ ]:
nltk.download('punkt')

In [ ]:
dataset=list()

In [ ]:
dataset = td ## dataset was set to be equal to "d" when this program was initially run with training data to check the accuracy using cross validation
## preprocessing the data to remove stop words, html tags and other punctuations
for i in range(len(dataset)):
    
    dataset[i] = dataset[i].lower()
    
    pattern = re.compile(r'\b(' + r'|'.join(stopwords.words('english')) + r')\b\s*')

    dataset[i] = re.sub(pattern,"",dataset[i])
    dataset[i] = re.sub(r'\W', ' ', dataset[i])
    dataset[i] = re.sub(r'<[a-b]+>', ' ', dataset[i])
    #dataset[i] = re.sub(r'', ' ', dataset[i])
    dataset[i] = re.sub(r'\s+', ' ', dataset[i])

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
## TF-IDF library used
vectorizer = TfidfVectorizer()


In [ ]:
## X has the TF-IDF for the training data. Dont run for test data run
X = vectorizer.fit_transform(dataset)
print(X.shape)

In [ ]:
## XT has the TF-IDF for the test data. Run this for test data run
XT = vectorizer.fit_transform(dataset)
print(XT.shape)

In [ ]:
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import random as sparse_random
##Dimensionality reduction using svd library
svd = TruncatedSVD(n_components=20, n_iter=7, random_state=42)


In [ ]:
## X has the reduced dimensionality for training data . Do not run for test data run
X=svd.fit_transform(X)


In [ ]:
## XT has the reduced dimensionlity for test data . Run for test data run.
XT=svd.fit_transform(XT)

In [ ]:
def weighted_knn(xTrain,y_train, xTest,k):
    xTrain=np.linalg.norm(xTrain,axis=1)#getting  unit vector + euclidean distance equivalent to cosine sim

    xTest=np.linalg.norm(xTest,axis=1)
    knn = KNeighborsClassifier(n_neighbors=k, weights='distance', algorithm='brute', leaf_size=30, p=2, metric='euclidean', metric_params=None, )
    
    knn.fit(X_train, y_train)    
    return knn.predict(X_test)

In [ ]:
#from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn import metrics
import matplotlib.pyplot as plt 

#will first check which is the best k
def pred(Ks,step):
    
    mean_acc = np.zeros((Ks-1))
    std_acc = np.zeros((Ks-1))
  
  
    for n in range(1,Ks,1):    
        
        yhat=weighted_knn(X_train,y_train,X_test,n)
        #yhat=weighted(X_train,y_train,X_test,n)
        mean_acc[n-1] = metrics.accuracy_score(y_test, yhat)    
        std_acc[n-1]=np.std(yhat==y_test)/np.sqrt(yhat.shape[0])

    print( "The best accuracy was:", np.round(mean_acc.max()*100,2), "% with k=", mean_acc.argmax()+1) 

    plt.plot(range(1,Ks),mean_acc,'g')
    plt.fill_between(range(1,Ks),mean_acc - 1 * std_acc,mean_acc + 1 * std_acc, alpha=0.05)
    plt.legend(('Accuracy ', '+/- 3xstd'))
    plt.ylabel('Accuracy ')
    plt.xlabel('Number of Neighbors (k)')
    plt.tight_layout()
    plt.show()

In [ ]:
import numpy as np
from sklearn.model_selection import KFold
## K fold cross validation done here . Do not run this for test data run
y = np.array(y.copy())
kf = KFold(n_splits=3,shuffle=True)
kf.get_n_splits(X)
Ks=30
step=1
print(kf)

for train_index, test_index in kf.split(X):
        print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = X_t[train_index], X_t[test_index]
        y_train, y_test = y[train_index], y[test_index]
        pred(Ks,step)

In [ ]:
xTest.shape

In [ ]:
## Run this for test data run
xTrain=X
xTest=XT
y_tain=y
xTrain=np.linalg.norm(xTrain,axis=1)

xTest=np.linalg.norm(xTest,axis=1)
knn = KNeighborsClassifier(n_neighbors=24, weights='distance', algorithm='brute', leaf_size=30, p=2, metric='euclidean', metric_params=None, )

knn.fit(xTrain, y_tain)    
   
result=knn.predict(xTest)

In [ ]:
len(result)

In [ ]:
## result file created here . Run this for test data run
f = open("result_tv_weighted_cosine.txt", "w")
for i in list(result):
    if i==1:
        f.write("+1\n")
    else:
        f.write("-1\n")
f.close()

In [ ]:
## do not run anything after this line
### These functions were tried and experimented with during but did not work as expected and was not able to debug them
### Finally used the knn library function to implement the algorithm
"""
import pandas as pd
import numpy as np
from scipy.spatial import distance
import sklearn
COSINE SIMILARITY
def knn(xTrain, xTest, k):
    """
    """
    Finds the k nearest neighbors of xTest in xTrain.
    Input:
    xTrain = n x d matrix. n=rows and d=features
    xTest = m x d matrix. m=rows and d=features (same amount of features as xTrain)
    k = number of nearest neighbors to be found
    Output:
    dists = distances between xTrain/xTest points. Size of n x m
    indices = kxm matrix with indices of yTrain labels
    """
    """
    #distances=sklearn.metrics.pairwise.cosine_similarity(xTrain, Y=xTest, dense_output=True)
    #print("cosine sim shape",distance.shape())
    dot = np.dot(xTrain, xTest.T)
    norma = np.linalg.norm(xTrain)
    normb = np.linalg.norm(xTest)
    distances = dot / (norma * normb)
    indices = np.argsort(distances, 0) #get indices of sorted items
    distances = np.sort(distances,0) #distances sorted in axis 0
    #returning the top-k closest distances.
    return indices[0:k, : ], distances[0:k, : ]"""

"""
import pandas as pd
import numpy as np

def knn(xTrain, xTest, k):
    """
    """Finds the k nearest neighbors of xTest in xTrain.
    Input:
    xTrain = n x d matrix. n=rows and d=features
    xTest = m x d matrix. m=rows and d=features (same amount of features as xTrain)
    k = number of nearest neighbors to be found
    Output:
    dists = distances between xTrain/xTest points. Size of n x m
    indices = kxm matrix with indices of yTrain labels
    """
    """
    #the following formula calculates the Euclidean distances.
    distances = -2 * xTrain@xTest.T + np.sum(xTest**2,axis=1) + np.sum(xTrain**2,axis=1)[:, np.newaxis]
    #because of numpy precision, some really small numbers might 
    #become negatives. So, the following is required.
    distances[distances < 0] = 0
    #for speed you can avoid the square root since it won't affect
    #the result, but apply it for exact distances.
    distances = distances**.5
    indices = np.argsort(distances, 0) #get indices of sorted items
    distances = np.sort(distances,0) #distances sorted in axis 0
    #returning the top-k closest distances.
    return indices[0:k, : ], distances[0:k, : ]""""

"""
def weighted_knn_predictions(xTrain,yTrain,xTest,k=3,weight=1):
    """
    """Input:
    xTrain = n x d matrix. n=rows and d=features
    yTrain = n x 1 array. n=rows with label value
    xTest = m x d matrix. m=rows and d=features
    k = number of nearest neighbors to be found
    Output:
    predictions = predicted labels, ie preds(i) is the predicted label of xTest(i,:)
    """
    """
    indices, distances = knn(xTrain,xTest,k)
    yTrain = yTrain.flatten()
    rows, columns = indices.shape
    predictions = list()
    w1=0
    w2=0
    for j in range(columns):
        temp = list()
        for i in range(rows):
            cell = indices[i][j]
            if yTrain[cell] == 1:
                w1+=distances[i][j]
            else:
                w2+=distances[i][j]
        if w1 >= w2:
            predictions.append(1) 
        else:
            predictions.append(-1)
    predictions=np.array(predictions)
    return predictions"""
"""
## count vectorizer was used before finalising with the tfid
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(dataset)
XT=vectorizer.fit_transform(dataset)
"""""

""""
##The bag of words model was also used with a good level of accuracy
# Creating the Bag of Words model
word2count = {}
for data in dataset:
    words = nltk.word_tokenize(data)
    for word in words:
        if word not in word2count.keys():
            word2count[word] = 1
        else:
            word2count[word] += 1
            """
"""
## 150 most used words were filtered from this before the TF-IDF approach
import heapq
freq_words = heapq.nlargest(150, word2count, key=word2count.get)
"""
"""
## with word tokenisation
X = []
for data in dataset:
    vector = []
    for word in freq_words :
        if word in nltk.word_tokenize(data):
            vector.append(1)
        else:
            vector.append(0)
    X.append(vector)
X = np.asarray(X)
""""